# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.<br>
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## Обзор данных


Импорти библиотеки pandas. Открытие данных из csv-файла в датафрейм и сохранение в переменную `data`. 

In [1]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')


In [28]:
#Вывод первых 20 строчек на экран
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [29]:
#Вывод основной информации о датафрейме
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21331 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21331 non-null  int64  
 1   days_employed          21331 non-null  float64
 2   dob_years              21331 non-null  int64  
 3   education              21331 non-null  object 
 4   education_id           21331 non-null  int64  
 5   family_status          21331 non-null  object 
 6   family_status_id       21331 non-null  int64  
 7   gender                 21331 non-null  object 
 8   income_type            21331 non-null  object 
 9   debt                   21331 non-null  int64  
 10  total_income           21331 non-null  int32  
 11  purpose                21331 non-null  object 
 12  total_income_category  21331 non-null  object 
 13  purpose_category       21331 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(7)
memory us

## Предобработка данных

### Удаление пропусков

In [30]:
#Вывод количество пропущенных значений для каждого столбца
data.isna().sum()

children                 0
days_employed            0
dob_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
total_income_category    0
purpose_category         0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. 

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [31]:
#Вывод медианного значения трудового стажа в днях
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.137274
компаньон            1555.947387
пенсионер          365286.622650
предприниматель       520.848083
сотрудник            1572.328285
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [32]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [33]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [34]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [35]:
data.isna().sum()

children                 0
days_employed            0
dob_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
total_income_category    0
purpose_category         0
dtype: int64

### Изменение типов данных

Замениv вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [38]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

В столбце `education`  есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [37]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.




In [39]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [40]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [41]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.



In [42]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [43]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследование данных и ответы на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Определим процент людей, которые имели задолженность по кредиту в каждой группе по количеству детей отдельно.

In [44]:
data_debt=data[['children','debt']]
data_pivot=data_debt.pivot_table(index=['children'],values='debt',aggfunc=('mean','count','sum')).reset_index()
data_pivot['mean']=data_pivot['mean']*100 
data_pivot





,children,count,mean,sum
0,0,14091,7.543822,1063
1,1,4808,9.234609,444
2,2,2052,9.454191,194
3,3,330,8.181818,27
4,4,41,9.756098,4
5,5,9,0.000000,0


На данный момент можно однозначно говорить о том, что наличие хотя бы одного ребенка в семье увеличивает риск задолженности по кредиту.
В виду значительной разницы между количеством людей с несколькими детьми, точно определить влияет ли увеличение количества детей на возврат кредита в срок исходя из имеющихся данных не представляется возможным, однако, сравнивая два относительно сопоставимых значения (1 ребенок в семье и 2 ребенка в семье), можно предположить, что увеличение количества детей в семье увеличивает риск невозврата должником кредита в срок.

Расчитаем зависимость между наличием хотя бы одного ребенка в семье и возвратом кредита в срок

In [45]:
people_with_children = data_debt.loc[data_debt['children'] > 0]
deb_with_children = people_with_children.loc[people_with_children['debt'] == 1]
deb_with_children_mean = len(deb_with_children) / len(people_with_children)

people_without_children = data_debt.loc[data_debt['children'] == 0]
deb_without_children = people_without_children.loc[people_without_children['debt'] == 1]
deb_without_children_mean = len(deb_without_children) / len(people_without_children)
if deb_with_children_mean>deb_without_children_mean:
    print ('Доля должников среди людей с детьми больше, чем без детей и составляет', deb_with_children_mean*100, '%')
else: 
    print ('Доля должников среди людей с без детей больше, чем с детьми и составляет', deb_without_children_mean*100, '%')

Доля должников среди людей с детьми больше, чем без детей и составляет 9.240331491712707 %


**Вывод:** Наличие детей у человека негативно влияет на возврат кредита в срок

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [46]:
data_status=data[['family_status','debt']]
data_status_grouped=data_status.groupby('family_status').sum()
data_status_grouped['count']=data_status.groupby('family_status').count()
data_status_grouped['mean']=data_status.groupby('family_status').mean()*100
data_status_grouped.sort_values(by='mean',ascending=False)

,debt,count,mean
family_status,,,
Не женат / не замужем,273,2796,9.763948
гражданский брак,385,4134,9.313014
женат / замужем,927,12261,7.560558
в разводе,84,1189,7.064760
вдовец / вдова,63,951,6.624606


**Вывод:** В группе людей с семейным положением не женат/ не замужем наибольший процент должников - 9,76%, наименьший процент должников в группе людей с семейным положением вдовец/вдова - 6,62%.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [47]:
data_income=data[['total_income','debt']]
data_income = data_income.loc[data_income['debt'] == 1]
data_income['debt_number']=data_income['total_income'].count()
data_income['income_group'] = data_income['total_income'].apply(categorize_income)
data_income_final=data_income[['income_group', 'debt','debt_number']].groupby('income_group').sum().sort_values('debt', ascending = False)# Ваш код будет здесь. Вы можете создавать новые ячейки.
data_income_final['mean']=data_income_final['debt']/data_income_final['debt_number']
data_income_final

,debt,debt_number,mean
income_group,,,
C,1353,2343396,0.000577
B,354,613128,0.000577
D,21,36372,0.000577
A,2,3464,0.000577
E,2,3464,0.000577


**Вывод:** Доля должников в каждой из групп одинакова, однако следует обратить внимание на размер каждой из групп. В данном случае имеет смысл сравнивать лишь относительно сопоставимые по размеру выборки, а именно группу "C" - доходность 50001–200000 и группу "B"- доходность 200001–1000000. Основываясь на полученных данных, можно предположить, что уровень дохода не влияет на возврат кредита в срок.

In [48]:
# Код ревьюера
data.groupby('total_income_category')['debt'].agg(['count', 'sum', 'mean'])\
  .sort_values(by = 'mean', ascending = False)

,count,sum,mean
total_income_category,,,
E,22,2,0.090909
C,15921,1353,0.084982
A,25,2,0.080000
B,5014,354,0.070602
D,349,21,0.060172


#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [49]:
data_purpose=data[['purpose_category','debt']]
data_purpose_grouped = data_purpose.groupby('purpose_category').count()
data_purpose_grouped['debt_count'] = data_purpose.loc[data_purpose['debt'] == 1].groupby('purpose_category').sum()
data_purpose_grouped['mean'] = data_purpose_grouped['debt_count'] / data_purpose_grouped['debt'] * 100
data_purpose_grouped
# Ваш код будет здесь. Вы можете создавать новые ячейки.

,debt,debt_count,mean
purpose_category,,,
операции с автомобилем,4279,400,9.347978
операции с недвижимостью,10751,780,7.255139
получение образования,3988,369,9.252758
проведение свадьбы,2313,183,7.911803


**Вывод:** Большая доля должников среди лиц, берущих кредит на операции с автомобилем - 9,347978% и на получение образования - 9.252758%, меньше всего должников, берущих кредит на операции с недвижимостью - 7.255139%

#### 3.5 Возможные причины появления пропусков в исходных данных.

В исходных данных пропуски содержались в данных о трудовом стаже и об уровне дохода.
Данные пропуски могли возникнуть в виду того, что за кредитом могли обращаться и безработные лица. Также необходимо учитывать возможность неофициального трудоустройства людей, в виду чего документально подтвердить свой доход и трудовой стаж они были не способны.
Также, следует отметить, что количество пропусков в двух разных колонках одинаково, это может свидетельствовать также и о программном сбое в определенный промежуток времени или о несовершенстве программы для внесения информации в целом, особенно учитывая, что большое количество данных в столбце со стажем имеет отрицательное значение. 

#### Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Медианное значение наиболее точно  отражает выборку при незаполненных данных. Среднее арифметическое отражает ситуацию некорректно, так как не учитывается разброс данных.

### Шаг 4: общий вывод.

В ходе исследования были проверены следующие зависимости:
1) Зависимость между количеством детей и возвратом кредита в срок.
По результатам исследования наличие детей у человека негативно влияет на возврат кредита в срок

2) Зависимость между семейным положением и возвратом кредита в срок.
Выявлено, что в группе людей с семейным положением не женат/ не замужем наибольший процент должников, наименьший процент должников в группе людей с семейным положением вдовец/вдова 

3) Зависимость между уровнем дохода и возвратом кредита в срок.
Наибольшее количество должников являются обладателями доходов 50001–200000, однако, если рассматривать долю должников среди всех кредиторов, то процентное соотношение в каждой из групп будет одинаково. Основываясь на полученных данных, можно предположить, что уровень дохода не влияет на возврат кредита в срок.

4) Зависимость цели кредита на его возврат в срок.
Большая доля должников среди лиц, берущих кредит на операции с автомобилем и на получение образования, меньше всего должников, берущих кредит на операции с недвижимостью 

Таким образом, наибольшая вероятность просрочки по кредиту возможна при совокупности вышеперечисленных факторов, а именно если кредит берет не женатый/ не замужний человек, имеющий детей, и берущий его на операции с автомобилем или на получение образование.

При этом наименьшая вероятность просрочки по кредиту возможна в случае, если кредит берет вдовец/вдова, не имеющий детей, и берущий его на операции с недвижимостью.

В качестве рекомендаций для повышения качества сбора данных можно рассмотреть вопрос о конкретизации данных по виду деятельности. Так, в предоставленных данных указаны лишь общие наименования: "студент", "сотрудник", "госслужащий", "предприниматель" и т.д. Если детализировать эти данные с указанием сферы, в которой данный человек трудоустроен, например "предприниматель в сфере общественного питания", это позволит выявить дополнительные группы риска при выдаче кредита.